# Using CLMBR to generate features and training models on those features

We can use a trained CLMBR model to generate features and then use those features in a logistic regression model.

In [1]:
import shutil
import os

TARGET_DIR = 'trash/tutorial_5'

if os.path.exists(TARGET_DIR):
    shutil.rmtree(TARGET_DIR)

os.mkdir(TARGET_DIR)

In [2]:
import femr.models.transformer
import pyarrow.csv
import datasets

# First, we compute our features

# Load some labels
labels = pyarrow.csv.read_csv('input/labels.csv').to_pylist()

# Load our data
dataset = datasets.Dataset.from_parquet("input/meds/data/*")

features = femr.models.transformer.compute_features(dataset,'input/clmbr_model', labels, num_proc=4, tokens_per_batch=128)

# We have our features
for k, v in features.items():
    print(k, v.shape)

Generating train split: 0 examples [00:00, ? examples/s]

Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

/share/pi/nigam/projects/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/site-packages/datasets/table.py:1357: FutureWarning: promote has been superseded by mode='default'.
  block_group = [InMemoryTable(cls._concat_blocks(list(block_group), axis=axis))]
/share/pi/nigam/projects/zphuo/miniconda3/envs/FEMR_ENV/lib/python3.10/site-packages/datasets/table.py:1383: FutureWarning: promote has been superseded by mode='default'.
  table = cls._concat_blocks(blocks, axis=0)
Some weights of the model checkpoint at input/clmbr_model were not used when initializing FEMRModel: ['task_model.final_layer.bias', 'task_model.final_layer.weight', 'transformer.in_norm.scale', 'transformer.layers.0.norm.scale', 'transformer.layers.1.norm.scale', 'transformer.out_norm.scale']
- This IS expected if you are initializing FEMRModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This I

<class 'femr.models.transformer.FEMRModel'> model type


Map (num_proc=4):   0%|          | 0/200 [00:00<?, ? examples/s]

ValueError: Couldn't find patient birthdate -- Patient has no birth events (which must be {meds.birth_code}): [{'time': datetime.datetime(1992, 1, 25, 0, 0), 'measurements': [{'code': 'SNOMED/184099003', 'text_value': None, 'numeric_value': None, 'datetime_value': None, 'metadata': None}, {'code': 'Gender/F', 'text_value': None, 'numeric_value': None, 'datetime_value': None, 'metadata': None}, {'code': 'Race/White', 'text_value': None, 'numeric_value': None, 'datetime_value': None, 'metadata': None}]}, {'time': datetime.datetime(1992, 2, 7, 0, 0), 'measurements': [{'code': 'ICD9CM/665.9', 'text_value': None, 'numeric_value': None, 'datetime_value': None, 'metadata': None}]}, {'time': datetime.datetime(1992, 4, 7, 0, 0), 'measurements': [{'code': 'ICD9CM/17', 'text_value': None, 'numeric_value': None, 'datetime_value': None, 'metadata': None}]}, {'time': datetime.datetime(1992, 5, 6, 0, 0), 'measurements': [{'code': 'ICD9CM/810.9', 'text_value': None, 'numeric_value': None, 'datetime_va

# Joining features and labels

Given a feature set, it's important to be able to join a set of labels to those features.

This can be done with femr.featurizers.join_labels

In [ ]:
import femr.featurizers

features_and_labels = femr.featurizers.join_labels(features, labels)

for k, v in features_and_labels.items():
    print(k, v.shape)

: 

# Data Splitting

When using a pretrained CLMBR model, we have to be very careful to use the splits used for the original model

In [ ]:
import femr.splits
import numpy as np

# We split into a global training and test set
split = femr.splits.PatientSplit.load_from_csv('input/clmbr_model/main_split.csv')

train_mask = np.isin(features_and_labels['patient_ids'], split.train_patient_ids)
test_mask = np.isin(features_and_labels['patient_ids'], split.test_patient_ids)

percent_train = .70
X_train, y_train = (
    features_and_labels['features'][train_mask],
    features_and_labels['boolean_values'][train_mask],
)
X_test, y_test = (
    features_and_labels['features'][test_mask],
    features_and_labels['boolean_values'][test_mask],
)

: 

# Building Models

The generated features can then be used to build your standard models. In this case we construct both logistic regression and XGBoost models and evaluate them.

Performance is perfect since our task (predicting gender) is 100% determined by the features

In [ ]:
import xgboost as xgb
import sklearn.linear_model
import sklearn.metrics
import sklearn.preprocessing

def run_analysis(title: str, y_train, y_train_proba, y_test, y_test_proba):
    print(f"---- {title} ----")
    print("Train:")
    print_metrics(y_train, y_train_proba)
    print("Test:")
    print_metrics(y_test, y_test_proba)

def print_metrics(y_true, y_proba):
    y_pred = y_proba > 0.5
    auroc = sklearn.metrics.roc_auc_score(y_true, y_proba)
    aps = sklearn.metrics.average_precision_score(y_true, y_proba)
    accuracy = sklearn.metrics.accuracy_score(y_true, y_pred)
    f1 = sklearn.metrics.f1_score(y_true, y_pred)
    print("\tAUROC:", auroc)
    print("\tAPS:", aps)
    print("\tAccuracy:", accuracy)
    print("\tF1 Score:", f1)


model = sklearn.linear_model.LogisticRegressionCV(penalty="l2", solver="liblinear").fit(X_train, y_train)
y_train_proba = model.predict_proba(X_train)[::, 1]
y_test_proba = model.predict_proba(X_test)[::, 1]
run_analysis("Logistic Regression", y_train, y_train_proba, y_test, y_test_proba)

: 